# 내가 원하는 카페의 모든 리뷰 크롤링하기
셀레니움 및 BeautifulSoup 둘 다를 활용한 코드입니다.  
jupyter notebook을 기준으로 하였습니다.
참고 : https://m.blog.naver.com/tamiblue/221723206818

단순 셀 실행 말고 무언가 바꿔야 하는 부분이 있는 경우,  
셀의 맨 첫 부분에 ### 표시를 해놓았습니다. 주의사항에 있는 사항들이 ### 표시 사항과 거의 같습니다.  

**주의사항**
1. 경로지정에 유의해주세요! chromedriver, publicdata_cafe.csv, 저장될 데이터의 위치들
2. [4] 실행한 뒤에 새로운 창을 한 번만 직접 클릭해주세요! 에러를 막을 수 있습니다.  
3. 각자 맡은 부분에 맞추어 for문의 range를 변경해서 진행해주시면 됩니다.
4. 한 번에 맡은 모든 부분을 for문에 넣기보다는 for문을 다른 셀들에서 돌려서 여러 파일로 나누어 저장하는 것을 추천드립니다. 

In [ ]:
#셀레늄 라이브러리 불러오기
from selenium import webdriver

#셀레늄에서 html문서가 생성될 때까지 대기하는 함수 불러오기
from selenium.webdriver.support.ui import WebDriverWait

#셀레늄에서 조건을 걸어줄 수 있는 함수 불러오기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

In [ ]:
### 크롬드라이버 경로 변경
path = '크롬드라이버 경로'

#드라이버 객체 생성 # 새로운 창 띄워주세요!
browser = webdriver.Chrome(path)

<ipython-input-2-45165c849dfc>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(path)


In [ ]:
###
# 필요한 함수들을 불러주고 데이터 가져오기
import re
import time
from pandas import DataFrame
import pandas as pd
import numpy as np


### 경로 변경 꼭 해주세요~~
df = pd.read_csv('소상공인데이터 저장경로\\publicdata_cafe.csv')
onlyname = df['이름']
location = df['법정동명']

## 본격적 크롤링

In [ ]:
### 아래 링크 실행 후 꼭 창으로 가서 클릭 한 번만 해주세요!
browser.get('https://map.kakao.com/')

아래의 결과물은 14000번째~14049번째 카페를 크롤링해온 결과입니다.
상황에 맞게 range 부분을 바꾸어 사용해주세요!

In [ ]:
### 7번째줄에 range 부분 변경 (그외에는 만지지 않고 그냥 돌려주시면 됩니다.)
# 이제 시작입니다.

start = time.time()
full = DataFrame(columns = ['name','crawlname','address','tags','user_id','user_name','user_review_num',
                            'user_rating_avg', 'rating', 'when','review'])


zeroreview = [] # review가 0인 경우의 index를 담기
nocafe = [] #카페를 찾을 수 없는 경우의 index를 담기
address_diff = [] #카페 리스트는 뜨지만(최대 12개까지 탐색), 주소지가 서로 다른 경우의 index를 담기


error = 0

### range 부분 바꿔주세요!!!
for i in range(len(onlyname)):
    
    name = onlyname[i]
    
    ## 카페 이름을 카카오맵에 입력
    browser.find_element(by = By.XPATH, value ='//*[@id="search.keyword.query"]').clear()
    browser.find_element(by = By.XPATH, value ='//*[@id="search.keyword.query"]').send_keys('%s'%name)
    
    time.sleep(1)
    browser.find_element(by = By.XPATH, value ='//*[@id="search.keyword.query"]').send_keys(Keys.ENTER)
        
    time.sleep(2)
    
    # 카카오맵에서 유사한 카페 리스트를 최대 12개를 보여줌
    # 가장 첫번째 것부터 시작해서 12번째 것까지 법정동이 일치하는 것이 있으면 그것의 상세정보를 누르게 됨.
    
    try:
        
        # 여기서 try except를 쓰는 것은, 카페 리스트가 1개일 때는 try 바로 다음의 것이,,
        # 카페리스트가 1개 초과일 때에는 except 다음의 것이 css selector이기 때문임.
        try:
            browser.find_element(by = By.CSS_SELECTOR, value ='#info\.search\.place\.list > li.PlaceItem.clickArea.PlaceItem-ACTIVE > div.info_item > div.contact.clickArea > a.moreview')
        except:
            browser.find_element(by = By.CSS_SELECTOR, value ='#info\.search\.place\.list > li:nth-child(1) > div.info_item > div.contact.clickArea > a.moreview')
        # 카페리스트가 0개라면 try except 둘 중 어느 것에도 해당되지 않기에, 더 큰 try에서 except로 빠져나가게 됨.
        
        
        # 카페리스트가 1개 이상인 경우, 최대 12개 중에서 법정동이 일치하는 것 찾는 코드
        try:
            isnt_location = True
            order = 0
            while isnt_location:
                order += 1
                try: # 카페리스트 2개 이상
                    address = browser.find_element(by=By.CSS_SELECTOR, value='#info\.search\.place\.list > li:nth-child(%d) > div.info_item > div.addr > p.lot_number'%order).text
                except: #카페리스트 딱 1개
                    address = browser.find_element(by = By.CSS_SELECTOR, value ='#info\.search\.place\.list > li.PlaceItem.clickArea.PlaceItem-ACTIVE > div.info_item > div.addr > p.lot_number').text
                
                # 지번주소에 법정동이 들어가있으면, 바로 그 카페가 우리가 찾는 카페임!
                if location[i] in address:
                    isnt_location = False
                    # #info\.search\.place\.list > li:nth-child(1) > div.info_item > div.contact.clickArea > a.moreview
                        
                else:
                    # 12개까지 봤는데 법정동이 없으면 그정도면 없는 거임..
                    if order==12:
                        print(i,'번째 카페',name,'은(는) 법정동명이 일치하지 않으므로 제외합니다.')
                        address_diff.append(i)
                        raise
        except:
            # 어차피 우리가 찾으려는 그 카페는 없기 때문에 뒤를 실행할 이유가 없으므로
            # 다음 for문을 돌리는 것이 이득임
            if order==12:
                continue
                
        # 이전에 언급한 것처럼, 요거프레소 쌍문점인데 봉구스 밥버거 쌍문점이 제일 먼저 나오고 법정동이 같기에 그냥 그의 정보를 수집하는 경우 발생 가능
        # 그래서 일단 crawlname이라는 변수로, 카카오맵에서 찾은 점포의 이름도 저장함.

        try:
            crawlname = browser.find_element(by=By.CSS_SELECTOR, value='#info\.search\.place\.list > li:nth-child(%d) > div.head_item.clickArea > strong > a.link_name > strong'%order).text
        except:
            crawlname = browser.find_element(by=By.CSS_SELECTOR, value='#info\.search\.place\.list > li.PlaceItem.clickArea.PlaceItem-ACTIVE > div.head_item.clickArea > strong').text
            
        #드디어 상세정보로 넘어가기

        try:
            browser.find_element(by = By.CSS_SELECTOR, value = '#info\.search\.place\.list > li:nth-child(%d) > div.info_item > div.contact.clickArea > a.moreview'%order).click()    
        except:
            browser.find_element(by = By.CSS_SELECTOR, value ='#info\.search\.place\.list > li.PlaceItem.clickArea.PlaceItem-ACTIVE > div.info_item > div.contact.clickArea > a.moreview').click()
            
        # 후기창으로 탭 전환

        browser.switch_to.window(browser.window_handles[1])
        time.sleep(3)
    
        
        # 태그도 코드 수정했어요...ㅠㅠ
        # 태그는 혹시나 content-based로 돌릴 일이 있을까봐 저장하도록 하게 함.
        
        try:
            tags = browser.find_element(by=By.CSS_SELECTOR, value='#mArticle > div.cont_essential > div.details_placeinfo > div:nth-child(4) > div > div > span').text
            # 가끔 이 위치가 전화번호 위치일 때가 있어서, 해시태그를 나타내는 #이 들어가있으면 tag, 아니면 전화번호.
            if '#' not in tags:
                tags = browser.find_element(by=By.CSS_SELECTOR, value='#mArticle > div.cont_essential > div.details_placeinfo > div:nth-child(5) > div > div > span').text
        except:
            # 정보 아예 없는 경우
            tags = np.nan
        
        # 사람들 별점 및 후기
        # 4명 이상이 후기를 작성한 경우 '후기 더보기'를 계속 눌러주어야 함
        more = True
        try:
            text = browser.find_element(by = By.CSS_SELECTOR, value='#mArticle > div.cont_evaluation > div.evaluation_review > a').text
            if text=='후기 더보기':
                # 후기 더보기가 안 나온다면, 그냥 이걸 지나가게 될 것임.(3개 이하)
                
                # 4개 이상의 경우 반복문을 통해 '후기 더보기' 버튼이 '후기 접기' 버튼이 될 때까지 눌러주어야 함.
                while(more):
                    time.sleep(1)
                    stop = browser.find_element(by = By.CSS_SELECTOR, value='#mArticle > div.cont_evaluation > div.evaluation_review > a').text
                    if stop !='후기 더보기':
                        more=False
                    else:
                        browser.find_element(by = By.CSS_SELECTOR, value='#mArticle > div.cont_evaluation > div.evaluation_review > a').click()
                    #text=='접기' 인 경우 그냥 넘어감
   
                    
        except:
            # 0개인 경우는 css selector로 찾아지지도 않음
            pass
        
        
        time.sleep(3)
        
        # 드디어 후기들을 모두 가져올 수 있게 됨. 
        # 하지만 우리 정확한 후기 개수를 알지 못하기에 또다시 반복문 시행
        
        # 그 전에 정보를 담을 list를 지정하기 (매 for문마다 list를 통해 dataframe 새로 만들 것이기에 여기에서 정의)
    
        t = 0
        loop=True
        useridlist = []
        usernamelist=[]
        userreviewnumlist=[]
        userrateavglist=[]
        ratinglist=[]
        whenlist=[]
        reviewlist=[]
        
        
        # find_element 하지 못한다는 것은(try문을 빠져나가 except로 빠진다는 것은), t번째 리뷰는 존재하지 않는다는 것임을 활용한 코드.
        # 반복문을 통해 t를 늘려감으로써 정보들을 축적
        while(loop):
            t=t+1
            try:
                userid = browser.find_element(by=By.CSS_SELECTOR, value='#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(%d)'%t).get_attribute('data-userid')
                username = browser.find_element(by=By.CSS_SELECTOR, value='#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(%d) > div.unit_info > a'%t).text
                userreviewnum = int(browser.find_element(by=By.CSS_SELECTOR, value='#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(%d) > div.unit_info > span:nth-child(5)'%t).text)
                userrateavg = float(browser.find_element(by=By.CSS_SELECTOR, value='#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(%d) > div.unit_info > span:nth-child(7)'%t).text)
                rating = int(browser.find_element(by=By.CSS_SELECTOR, value ='#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(%d) > div.star_info > div > span > span'%t).get_attribute('style')[7:-2])/20
                when = browser.find_element(by=By.CSS_SELECTOR, value='#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(%d) > div.unit_info > span.time_write'%t).text
                try:
                    review = browser.find_element(by=By.CSS_SELECTOR, value='#mArticle > div.cont_evaluation > div.evaluation_review > ul > li:nth-child(%d) > div.comment_info > p'%t).text
                except: 
                    review= np.nan
        
                useridlist.append(userid)
                usernamelist.append(username)
                userreviewnumlist.append(userreviewnum)
                userrateavglist.append(userrateavg)
                ratinglist.append(rating)
                whenlist.append(when)
                reviewlist.append(review)
        
            except:
                # 더이상의 리뷰가 없는 경우 여기로 오게 됨.
                if t == 1:
                    zeroreview.append(i)
                    
                print(i, '번째 카페', name,'에 대해','총 %d개의 리뷰가 저장되었습니다.'%(t-1))
                loop=False
        
        # 쌓아놓은 list들을 dataframe으로
        one = pd.DataFrame({'name':name,'crawlname':crawlname,'address':location[i],'tags':tags,
                            'user_id':useridlist,'user_name':usernamelist,'user_review_num':userreviewnumlist,
                            'user_rating_avg':userrateavglist, 'rating': ratinglist, 'when':whenlist,'review':reviewlist})
        
        # 이전 for문에서 만들어진 것에, 새로 만들어지는 것을 덧붙이는 방식.
        full = pd.concat([full, one],ignore_index = True)
        
        
        # 데이터 축적이 끝났기 때문에, 사용자 후기 탭을 닫아주고 원래 카카오맵 탭으로 돌아감.
        browser.close()
        browser.switch_to.window(browser.window_handles[0])
        
    except:
        # 이것은, 카페를 검색했을 때 리스트도 뜨지 않는 경우에 해당함.
        
        error = error+1
        nocafe.append(i)
        print(i, '번째 카페', name,'은(는) 폐업했거나 카카오맵에 정보가 없습니다.')
    
    if i%10==0:
        print(i+1,'개의 카페 크롤링 완료')
        # 혹시 몰라서 한 번 새로고침 하는 느낌으로다가..
        browser.get('https://map.kakao.com/')
        time.sleep(1)
        
end = time.time()
print(f"{end - start:.5f} sec")


14000 번째 카페 까페뉴 에 대해 총 1개의 리뷰가 저장되었습니다.
14001 개의 카페 크롤링 완료
14001 번째 카페 까소엔더카페 더카페 에 대해 총 6개의 리뷰가 저장되었습니다.
14002 번째 카페 미완성식탁 에 대해 총 64개의 리뷰가 저장되었습니다.
14003 번째 카페 리더스 은(는) 법정동명이 일치하지 않으므로 제외합니다.
14004 번째 카페 한잎 에 대해 총 0개의 리뷰가 저장되었습니다.
14005 번째 카페 스타벅스마포아크로타워점 마포아크로타워점 에 대해 총 15개의 리뷰가 저장되었습니다.
14006 번째 카페 오늘도,카페일리터 은평점 에 대해 총 0개의 리뷰가 저장되었습니다.
14007 번째 카페 이디야커피 보라매점 은(는) 법정동명이 일치하지 않으므로 제외합니다.
14008 번째 카페 성화커피전문점 은(는) 폐업했거나 카카오맵에 정보가 없습니다.
14009 번째 카페 생명나무 에 대해 총 0개의 리뷰가 저장되었습니다.
14010 번째 카페 옹기종기 은(는) 법정동명이 일치하지 않으므로 제외합니다.
14011 번째 카페 스타벅스 서울아트센터점 에 대해 총 13개의 리뷰가 저장되었습니다.
14012 번째 카페 유제니 은(는) 법정동명이 일치하지 않으므로 제외합니다.
14013 번째 카페 라테라세 은(는) 법정동명이 일치하지 않으므로 제외합니다.
14014 번째 카페 제주스 은(는) 법정동명이 일치하지 않으므로 제외합니다.
14015 번째 카페 낙랑파라 아현점 에 대해 총 16개의 리뷰가 저장되었습니다.
14016 번째 카페 카페희다 에 대해 총 5개의 리뷰가 저장되었습니다.
14017 번째 카페 나드카페 에 대해 총 2개의 리뷰가 저장되었습니다.
14018 번째 카페 메가엠지씨커피 회기역점 에 대해 총 14개의 리뷰가 저장되었습니다.
14019 번째 카페 아이갓에브리씽 국회도서관점 은(는) 법정동명이 일치하지 않으므로 제외합니다.
14020 번째 카페 카페 오솔식물 에 대해 총 2개의 리뷰가 저장되었습니다.
14021 개의 카페 크롤링

In [ ]:
# 이런식으로 잘 되었나 한 번 확인해주세요~
full

,name,crawlname,address,tags,user_id,user_name,user_review_num,user_rating_avg,rating,when,review
0,스타벅스 용산해링턴스퀘어점,스타벅스 용산해링턴스퀘어점,한강로3가,1522-3232 대표번호,1ufmf0o,마리오,73,4.5,5.0,2022.04.02.,오늘도 방문하고 갑니다.
1,스타벅스 용산해링턴스퀘어점,스타벅스 용산해링턴스퀘어점,한강로3가,1522-3232 대표번호,mv6nug,야호,43,3.5,3.0,2022.03.03.,케이크 포장해왔는데 빵이 푸석푸석 ㅠㅠ 결국 버렸네요.. 확인 잘 해주시고 판매해주...
2,스타벅스 용산해링턴스퀘어점,스타벅스 용산해링턴스퀘어점,한강로3가,1522-3232 대표번호,td58pn,❛ ֊ ❛,8,4.3,5.0,2021.12.30.,너무 친절하시고 자리가 많아서 좋습니다.
3,스타벅스 용산해링턴스퀘어점,스타벅스 용산해링턴스퀘어점,한강로3가,1522-3232 대표번호,gnqlbe,이용욱,730,4.4,5.0,2021.12.19.,
4,스타벅스 용산해링턴스퀘어점,스타벅스 용산해링턴스퀘어점,한강로3가,1522-3232 대표번호,1a85dbr,펠릭스,41,4.4,5.0,2021.08.05.,직원분들 친절하고 커피 맛도 좋음
...,...,...,...,...,...,...,...,...,...,...,...
646,AID카페 롯데월드몰점,AID카페 롯데월드몰점,신천동,02-3213-4548 대표번호,10sovn7,WH,69,4.2,3.0,2021.06.06.,에이드는 그냥저냥.. 케이크는 생각보다 무난..
647,AID카페 롯데월드몰점,AID카페 롯데월드몰점,신천동,02-3213-4548 대표번호,g7mcp3,뚜뚜,68,3.9,1.0,2019.07.16.,비싸고 의자가 불편하고 와이파이 안돼요
648,달코롬,달코롬,미아동,NaN,194rev7,임정택,1,5.0,5.0,2020.10.17.,분위기 가성비 갑 커피한잔 땡길때 강추
649,쥬청과 강남세곡점,쥬청과 강남세곡점,자곡동,#과일카페 #제로페이,1a7t7p7,.,29,2.0,1.0,2021.08.08.,주인아저씨 매번 마스크 안쓰고 기침하세요...한두번도 아니고 더러워서 못가겠어요...


In [ ]:
### 경로 지정 조심해주세요!!
# 다 되면 dataframe을 csv로 저장하기
# for문을 어디에 대해서 시행했는지 0-10처럼 옆에 적어주시면 데이터를 합칠 때 훨씬 좋을 것 같습니다!


full.to_csv('저장경로/파일이름.csv')

In [ ]:
# 문제가 있는 (?) 카페가 무엇인지 기존데이터(소상공인)의 인덱스들도 확인해보기
print(zeroreview)
print(nocafe)
print(address_diff)

# 확 줄었네요...!

[14004, 14006, 14009, 14030, 14038, 14040]
[14008, 14025, 14037, 14045]
[14003, 14007, 14010, 14012, 14013, 14014, 14019, 14024, 14042]


In [ ]:
print('제대로 크롤링 안 된 애들 수')
print(len(nocafe) + len(address_diff))

제대로 크롤링 안 된 애들 수
13


In [ ]:
# 크롤링되지 않은 경우들 저장
indexdict = dict({'zeroreview': zeroreview,
     'nocafe':nocafe,
     'address_diff':address_diff})

In [ ]:
### 경로지정
# 크롤링되지 않은 경우 list 저장하기

import pickle
with open('C:\\Users\\lsj32\\Desktop\\DSL\\DSL_2\\modeling\\crawling\\index_dict14000-14050.pkl','wb') as f:
    pickle.dump(indexdict,f)

In [ ]:
# 창 닫겠습니다! 드디어 끝!
browser.quit()